<center style="font-size: 16px; font-weight: normal">
  `21 Microservices Architecture Patterns Study</center>

<center><h1>Chapter 4. Our First Use Case:<br/> Flask API and Service Layer</h1></center>

<center>
  Joseph Kim &lt;cloudeyes@gmail.com&gt;<br/>
  Dec 28. 2020<br/>
</center>

<center>
  <img src="https://images-na.ssl-images-amazon.com/images/I/51-q9CWNBHL._SX379_BO1,204,203,200_.jpg" style="width: 200px">
</center>

<center>
  <a href="https://github.com/cloudeyes/2021-msa-study/blob/main/04-flask-api-and-service-layer/04-flask-api-and-service-layer.ipynb">Download Jupyter Notebook</a>
</center>

## Introduction

In this chapter, we discuss:
- the differences between *orchestration logic*, *business logic*, and *interfacing code*.
- the *Service Layer pattern*
  - to take care of orchestrating our workflows
  - and defining the use cases of our system.
  
We'll also discuss "testing" again:
- by combining the Service Layer with our repository abstraction 


**Overal structure:**

<img src="./images/fig4-2.png" width="500px"/>

## Connecting Our Application to the Real World

**Strategy:**

1. Quickly make a MVP(Minimum-Valuable Product)
1. And then refactor toward a cleaner architecture.

**Steps:**

1. **Use Flask to put an API endpoint:** 
  - in front of our allocate domain service. 
  - wire up the database session and our repository. 
  - test it with an end-to-end test with quick-and-dirty test data.
1. **Refactor out a service layer:** 
  - can serve as an abstraction to capture the use case 
  - sits between Flask and our domain model. 
  - tests and show how they can use `FakeRepository`.
1. **Experiment with different types of parameters:** 
  - using primitive data types 
  - allows the service layer be decoupled from the model layer.

## A First End-to-End Test

- https://flask.palletsprojects.com/en/1.1.x/testing/

### Prerequisites

In [7]:
!pip install flask | grep "satisfied: flask"

In [5]:
!tree app -I "__pycache__"

app
├── __init__.py
├── models.py
├── orm.py
├── repository.py
├── services.py
├── tests
│   ├── __init__.py
│   └── test_models.py
└── :w

1 directory, 8 files


In [2]:
from flask import Flask, jsonify, request
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from app import models, orm, repository
from app.repository import SqlAlchemyRepository
#from app import config
#from app import repository

metadata = orm.start_mappers()
engine = orm.init_engine(metadata, "sqlite://", show_log=True)
get_session = orm.sessionfactory(engine) 
c = lambda: SqlAlchemyRepository(get_session())

CREATE TABLE order_line (
	id INTEGER NOT NULL, 
	sku VARCHAR(255), 
	qty INTEGER NOT NULL, 
	orderid VARCHAR(255), 
	PRIMARY KEY (id)
)

CREATE TABLE batch (
	id INTEGER NOT NULL, 
	reference VARCHAR(255), 
	_purchased_quantity INTEGER, 
	sku VARCHAR(255), 
	eta DATE, 
	PRIMARY KEY (id), 
	UNIQUE (reference)
)

CREATE TABLE allocation (
	orderline_id INTEGER NOT NULL, 
	batch_id INTEGER NOT NULL, 
	PRIMARY KEY (orderline_id, batch_id), 
	FOREIGN KEY(orderline_id) REFERENCES order_line (id), 
	FOREIGN KEY(batch_id) REFERENCES batch (id)
)




먼저 실패하는 테스트를 작성하세요.

In [6]:
from app import mytest

In [8]:
mytest.init(__name__)

In [10]:
import uuid

In [12]:
uuid.uuid4()

UUID('b71d5a3a-bc24-47ae-8a31-b2e25dade93c')

In [13]:
str(uuid.uuid4())

'4158dde3-f222-4151-a2bc-d9a7d679045e'

In [9]:
@mytest.unit
def test_api_returns_allocation(add_stock):
    sku, othersku = random_sku(), random_sku('other')
    earlybatch = random_batchref(1)
    laterbatch = random_batchref(2)
    otherbatch = random_batchref(3)
    add_stock([
        (laterbatch, sku, 100, '2011-01-02'),
        (earlybatch, sku, 100, '2011-01-01'),
        (otherbatch, othersku, 100, None),
    ])
    data = {'orderid': random_orderid(), 'sku': sku, 'qty': 3}
    url = config.get_api_url()
    r = requests.post(f'{url}/allocate', json=data)
    assert r.status_code == 201
    assert r.json()['batchref'] == earlybatch


Traceback (most recent call last):
  File "/home/ykkim/notebooks/2021-msa-study/04-flask-api-and-service-layer/app/__init__.py", line 39, in unit
    try:
AssertionError: name "add_stock" should be defined in the module


In [3]:
app = Flask(__name__)

@app.route("/allocate", methods=['POST'])
def allocate_endpoint():
    with get_repo() as repo:
        batches = repo.list()
        line = models.OrderLine(
            request.json['orderid'],
            request.json['sku'],
            request.json['qty'],
        )
        batchref = models.allocate(line, batches)

    return jsonify({'batchref': batchref}), 201

## The Straightforward Implementation

## Error Conditions That Require Database Checks

## Introducing a Service Layer, and Using FakeRepository to Unit Test It

### A Typical Service Function

## Why Is Everything Called a Service?

## Putting Things in Folders to See Where It All Belongs

## Wrap-up